# 데모 프로그램

In [1]:
'''
- tkinter 간략한 강좌
http://pythonstudy.xyz/python/article/120-Tkinter-%EC%86%8C%EA%B0%9C

- 컬러 차트
http://www.science.smith.edu/dftwiki/index.php/Color_Charts_for_TKinter
'''
import tkinter
import tkinter.font
import cv2
import numpy as np
import dlib
import PIL.Image, PIL.ImageTk
import time
from keras.models import load_model
from imutils.face_utils import FaceAligner

class App:
    def __init__(self, window, window_title, video_source=0):
        self.thresh = 0.5 # 값 범위: 0.0 ~ 1.0
        
        self.win_width = 1000
        self.win_height = 600
        self.bDetect = False
        self.bPause = False
        
        self.window = window
        self.window.title(window_title)
        self.window.geometry("%dx%d+0+0" % (self.win_width, self.win_height))
        self.window.resizable(False, False)
        self.video_source = video_source

        ### open video source (by default this will try to open the computer webcam)
        self.vid = MyVideoCapture(self.video_source)

        ### Create a canvas that can fit the above video source size
        self.wholewidth = int(self.win_height * 0.76)
        self.wholeheight = int(self.wholewidth * (480 / 640))
        self.facewidth = int(self.win_height * 0.25)
        
        # frame
        img_faceframe = PIL.ImageTk.PhotoImage(file='./rc/faceframe.png')
        self.lbl_faceframe = tkinter.Label(image=img_faceframe)
        self.lbl_faceframe.image = img_faceframe
        img_frameframe = PIL.ImageTk.PhotoImage(file='./rc/frameframe.png')
        self.lbl_frameframe = tkinter.Label(image=img_frameframe)
        self.lbl_frameframe.image = img_frameframe
        
        self.lbl_faceframe.place(relx=0.02,rely=0.03)
        self.lbl_frameframe.place(relx=0.47,rely=0.03)
        
        
        # label
        img_logo = PIL.ImageTk.PhotoImage(file='./rc/logo.png')
        self.lbl_logo = tkinter.Label(image=img_logo)
        self.lbl_logo.image = img_logo
        self.lbl_logo.place(relx=0.47,rely=0.73)
        
        font = tkinter.font.Font(family="Times", size=16)
        self.lbl_text1 = tkinter.Label(text="DETECTION", width=12, font=font, bg='gray75')
        self.lbl_text2 = tkinter.Label(text="RESULT", width=12, font=font, bg='gray75')
        self.lbl_text1.place(relx=0.07, rely=0.13)
        self.lbl_text2.place(relx=0.26, rely=0.13)
        
        font2 = tkinter.font.Font(family="Times", size=14)
        self.lbl_preds = []
        self.lbl_preds.append(tkinter.Label(text="", width=15, height=2, font=font2, bg='gray75'))
        self.lbl_preds.append(tkinter.Label(text="", width=15, height=2, font=font2, bg='gray75'))
        self.lbl_preds[0].place(relx=0.26, rely=0.33)
        self.lbl_preds[1].place(relx=0.26, rely=0.65)
        self.lbl_msgs = []
        self.lbl_msgs.append(tkinter.Label(text="", width=15, height=2, font=font2, bg='gray75'))
        self.lbl_msgs.append(tkinter.Label(text="", width=15, height=2, font=font2, bg='gray75'))
        self.lbl_msgs[0].place(relx=0.26, rely=0.40)
        self.lbl_msgs[1].place(relx=0.26, rely=0.72)
                            

        # video
        self.canvas_faces = []
        self.canvas_faces.append(tkinter.Canvas(window, width = self.facewidth, height = self.facewidth, highlightthickness=0))
        self.canvas_faces.append(tkinter.Canvas(window, width = self.facewidth, height = self.facewidth, highlightthickness=0))
        self.canvas_frame = tkinter.Canvas(window, width = self.wholewidth, height = self.wholeheight, highlightthickness=0)
        
        self.canvas_faces[0].place(relx=0.07, rely=0.25)
        self.canvas_faces[1].place(relx=0.07, rely=0.57)
        self.canvas_frame.place(relx=0.495, rely=0.07)

        # Button
        font2 = tkinter.font.Font(family="Times", size=10)
        self.btn_pause=tkinter.Button(window, text="Pause", font=font2, width=20, borderwidth=3, command=self.func_pause, bg="white", activebackground='cornflower blue', highlightbackground='blue', relief='groove')
        self.btn_pause.place(relx=0.57, rely=0.66)
        
        self.btn_detect=tkinter.Button(window, text="Detect", font=font2, width=20, borderwidth=3, command=self.func_detect, bg="white", activebackground='cornflower blue', highlightbackground='blue', relief='groove')
        self.btn_detect.place(relx=0.75, rely=0.66)

        # After it is called once, the update method will be automatically called every delay milliseconds
        self.delay = 15
        self.update()

        print('>> Running..')
        self.window.mainloop()

    def func_pause(self):
        self.bPause = not self.bPause
        if self.bPause:
            self.btn_pause['bg'] = "cornflower blue"
        else:
            self.btn_pause['bg'] = "white"
    
    def func_detect(self):
        if not self.bPause:
            self.bDetect = not self.bDetect
            if self.bDetect:
                self.btn_detect['bg'] = "cornflower blue"
            else:
                self.btn_detect['bg'] = "white"
        else:
            print("[ERROR] Can't detect the faces when the button 'pause' pressed")
    
            
    def update(self):
        ret, frame, rects, aligns = self.vid.detect_faces()
        if ret and (frame is not None):
            if not self.bPause:
                faces = []
                faces.append(cv2.imread('./rc/empty.png'))
                faces.append(cv2.imread('./rc/empty.png'))
                
                if self.bDetect:
                    for i in range(2):
                        self.lbl_preds[i].config(text="", bg='gray75')
                        self.lbl_msgs[i].config(text="")
                    
                    if rects:
                        for i, (rect, aligned) in enumerate(zip(rects, aligns)):
                            msg, (ys,ye,xs,xe) = self.vid.check_face(frame, rect)
                            size = xe-xs
                            
                            # 얼굴 영역만 자르기
                            faces[i] = frame[ys:ye, xs:xe].copy()
                            
                            if msg == "":
                                score = self.vid.detect_spoof(aligned)

                                if score > self.thresh:   color = (0,255,0); pred = "live"; bg = 'green2'
                                else:                     color = (255,0,0); pred = "fake"; bg = 'red'
                                    
                            else:
                                color = (255,192,0); pred = "undetectable"; bg = 'orange'

                            font = cv2.FONT_HERSHEY_SIMPLEX
                            textsize = cv2.getTextSize(pred, font, 1, 2)[0]
                            textX = xs + (size - textsize[0]) // 2
                            cv2.putText(frame, pred, (textX, ys-20), font, 1, color, 2)

                            cv2.rectangle(frame, (xs,ys), (xe,ye), color, 2)
                            
                            self.lbl_preds[i].config(text=pred, bg=bg)
                            self.lbl_msgs[i].config(text=msg)
            
                    
                # 비디오 위젯 업데이트
                self.faces = [None, None]
                for i in range(2):
                    faces[i] = cv2.resize(faces[i], (self.facewidth, self.facewidth))
                    self.faces[i] = PIL.ImageTk.PhotoImage(image = PIL.Image.fromarray(faces[i]))
                    self.canvas_faces[i].create_image(0, 0, image = self.faces[i], anchor = tkinter.NW)

                frame = cv2.resize(frame, (self.wholewidth, self.wholeheight))
                self.frame = PIL.ImageTk.PhotoImage(image = PIL.Image.fromarray(frame))
                self.canvas_frame.create_image(0, 0, image = self.frame, anchor = tkinter.NW)
                
        self.window.after(self.delay, self.update)

            
            
class MyVideoCapture:
    def __init__(self, video_source=0, facesize=256):
        # Open the video source
        self.cap = cv2.VideoCapture(video_source)
        if not self.cap.isOpened():
            raise ValueError("Unable to open video source", video_source)

        self.width = self.cap.get(cv2.CAP_PROP_FRAME_WIDTH)
        self.height = self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
        
        self.detector = dlib.get_frontal_face_detector()
        self.predictor = dlib.shape_predictor('./trained_model/shape_predictor_5_face_landmarks.dat')
        self.fa = FaceAligner(self.predictor, desiredLeftEye=(0.35, 0.35), desiredFaceWidth=facesize)
        
        self.model = load_model('./trained_model/NUAA+CASIA-22-0.2004.hdf5')
        print('>> model loaded.')

        
    def detect_faces(self):
        ret, frame = self.cap.read()
        if not ret:
            return (ret, None, None, None)
        
        frame = cv2.flip(frame, 1)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        rects = self.detector(frame)
        
        if len(rects) > 0:
            rects = sorted(rects, key=lambda x:x.area(), reverse=True)

            faces = []
            aligns = []
            for i, rect in enumerate(rects):
                if i>=2: break

                xs, ys, xe, ye = rect.left(), rect.top(), rect.right(), rect.bottom()
                w,h = xe-xs, ye-ys

                if xs < 0 or xs >= self.width or ys < 0 or ye >= self.height or w <= 0 or h <= 0:
                    continue
                
                aligned = self.fa.align(frame, gray, rect)
                
                faces.append(rect)
                aligns.append(aligned)
                
            return (ret, frame, faces, aligns)

        else:
            return (ret, frame, None, None)
            
            
    def detect_spoof(self, img, th=0.5):
        img = img.astype('float32') / 255.0
        score = self.model.predict(np.expand_dims(img, axis=0)).squeeze()
        return score
    
    def check_face(self, img, rect):
        # 얼굴 특징점 검출
        shape = self.predictor(img, rect)
        rEye = ((shape.part(0).x + shape.part(1).x) // 2, (shape.part(0).y + shape.part(1).y) // 2)
        lEye = ((shape.part(2).x + shape.part(3).x) // 2, (shape.part(2).y + shape.part(3).y) // 2)
        nose = (shape.part(4).x, shape.part(4).y)

        # 얼굴 위치 및 길이 정보 계산
        xs, xe, ys, ye = rect.left(), rect.right(), rect.top(), rect.bottom()
        l = max(xe-xs,ye-ys)
        
        # 얼굴 제약 조건 검사
        if self.reject_ipd(rEye, lEye, nose, self.width, muRatio=15.0):
            return 'More closer', (ys, ys+l, xs, xs+l)
        
        if self.reject_angle(rEye, lEye, nose):
            return "Keep your head\nstraight", (ys, ys+l, xs, xs+l)
        
        return "", (ys, ys+l, xs, xs+l)
        
    
    def reject_ipd(self, lEye, rEye, nose, width, muRatio=28.8, rhoRatio=3.6, alpha=2.0):
        '''
        @ brief
            Check IPD(inter pupillary distance) in the face

        @ params
        img: numpy array
            input image
        width: int
            width of the input image
        muRatio: float (default=28.8)
            mean of the IPD ratio for reliable face spoofing detection
        rhoRatio: float (default=3.6)
            standard devidation of the IPD ratio
        alpha: float (default=2.0)
            desired valid IPD range level
        '''
        ipd = abs(rEye[0]-lEye[0])

        mu = width * muRatio / 100    # Ratios are based on the width of the input image
        rho = width * rhoRatio / 100
        
        ## Reject or pass
        if abs(ipd - mu) > alpha * rho:
            return True
        else:
            return False

        
    def reject_angle(self, lEye, rEye, nose, rollThres=10, yawThres=0.1):
        '''
        @ brief
            Check the degree of rotation of the face

        @ params
        info: dict
            dict of the face rectangle and landmarks
        rollThres: int (default=10)
            threshold of the roll rotation (unit: degree)
        yawThres: float (default=0.1)
            threshold of the yaw rotation
        '''
        ## Calulate angle of roll rotation(=in-plain rotation) of input face
        slope = (rEye[1] - lEye[1]) / (rEye[0] - lEye[0] + 1e-7)
        rad = np.arctan(slope)
        deg = np.degrees(rad)
        roll = abs(deg)
        
        ## Calculate ratio of yaw rotation of input face
        eye2eyeVec = np.array([rEye[0]-lEye[0], rEye[1]-lEye[1]])             # eye2eye vector
        eye2noseVec = np.array([nose[0]-lEye[0], nose[1]-lEye[1]])            # eye2nose vector
        eye2eyeNorm = np.sqrt(np.dot(eye2eyeVec, eye2eyeVec))                 # norm of the eye2eye vector
        projVecNorm = np.dot(eye2eyeVec, eye2noseVec) / (eye2eyeNorm + 1e-7)  # norm of the vector projected eye2nose vector into eye2eye vector
        yaw = abs(projVecNorm/(eye2eyeNorm + 1e-7) - 0.5)                     # ratio between eye2eye distance and eye2nose distance

        ## Reject or pass
        if roll > rollThres or yaw > yawThres:
            return True
        else:
            return False
        
    
    # Release the video source when the object is destroyed
    def __del__(self):
        if self.cap.isOpened():
            self.cap.release()

if __name__ == '__main__':
    # Create a window and pass it to the Application object
    App(tkinter.Tk(), "SMU - Face Spoofing Detector")

Using TensorFlow backend.














Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


>> model loaded.
>> Running..
